Version 5. The intent here is to create two query focussed data sets from the original event data. Each should be indexed in such a way to enable easy analysis of the data whilst containing all relevant data. The two data sets will be:
<ul><li>Missions: New state every time a mission changes orbit and if a component is added or removed. To include a List of travellers and craft for every state. Consider collapsing state changes that occur at the same time.</li>
    <li>Components: New state every time the component changes mission and if its mission changes orbit. To include the orbit of the mission.</li></ul>

Spaceflight data.
Some questions to ask (<em>Emphasised</em> text indicates as yet unanswered questions): 
<ul>
    <li>Who's been blasted into space the most times?</li>
    <li>Who's been to the most different orbits?</li>
    <li>Which spacecraft has been blasted into space the most times? And how many have been used more than once?</li>
    <li>Who's spent the most time in space?</li>
    <li>Which re-usable spacecraft has been used the most frequently?</li>
    <li>What is the largest number of people to have been on the same spacecraft/mission at the same time? <em>(and when was this?)</em></li>
    <li><em>What is the largest number of free-flying missions to be in orbit at the same time? (and when was this?)</em></li>
</ul>

In [1]:
import numpy as np
import pandas as pd

Read in data file

In [2]:
el = pd.read_csv("data/sftl-2020.txt", sep="\t", header=0, parse_dates=[0], dtype={'eventType': 'category'})
el.tail()

,date,subject,eventType,object
5320,2020-04-17,Andrew_R._Morgan,JOINS,Soyuz_MS-15
5321,2020-04-17,Oleg_Skripochka,JOINS,Soyuz_MS-15
5322,2020-04-17,Jessica_Meir,JOINS,Soyuz_MS-15
5323,2020-04-17,Soyuz_MS-15,DEPARTS,LEO
5324,2020-04-17,Soyuz_MS-15,ARRIVES,Earth


Some explanation of the data.</br>
The events describe changes to the state of three and a bit different types of object. The three main types are Mission, Orbit and Component. A mission is a slightly abstract concept that groups together the components for a particular purpose. All the components of a mission are physically connected and share a set of orbital ephemeris. An orbit is a rough grouping of different orbits. E.g. LEO for all low-Earth-orbits. Components are further sub-types into travellers and spacecraft. Travellers are people and animals that have been sent into space. Spacecraft are the ships that carried them. The eventType indicates the type of objects in question as follows:
<ul>
    <li>ARRIVES, DEPARTS and ENDS: The subject of the event is a mission, the object is an orbit.</li>
    <li>JOINS: The subject is a traveller and the object is a mission.</li>
    <li>SUPPORTS: The subject is a spacecraft and the object is a mission.</li>
</ul>  
Note that missions can exist without any craft assigned to them. This is typical for missions to space stations whereby the spacecraft becomes part of the space station mission but it's original mission continues until such time as the spacecraft un-docks and returns to Earth.</br>
Timestamps are only accurate to the nearest day but the order that the events occur is cronologically correct (sort of) and therefore shouldn't be ignored.</br>
I can't remember what BREAK events are for. I think it might be to separate groups of events that happen on the same day.

Pull out all of the unique components and set up a table to track their state.

In [3]:
componentevents = el[(el['eventType']=="SUPPORTS")|(el['eventType']=="JOINS")]
c = componentevents[["subject","eventType","object"]].groupby(by=["subject","eventType"], observed=True).count()
c.reset_index("eventType", inplace=True)
c["Type"]=c["eventType"].map({'SUPPORTS': 'craft', 'JOINS': 'traveller'})
c.drop("eventType", axis=1, inplace=True)
c.rename(columns={"object": "EventCount"}, inplace=True)
c.index.rename("component", inplace=True)
c["mission"] = None
c["currentstate"] = None
c

,EventCount,Type,mission,currentstate
component,,,,
7K-L1 K146,1,craft,None,None
7K-L1 K154,1,craft,None,None
7K-L1 Z4,1,craft,None,None
Abdul_Ahad_Mohmand,3,traveller,None,None
Aidyn_Aimbetov,3,traveller,None,None
...,...,...,...,...
Zarya,1,craft,None,None
Zhai_Zhigang,1,traveller,None,None
Zhang_Xiaoguang,3,traveller,None,None


So, I'm a bit concerned that the data set may have a lot of duplicate names. All of the names are drawn from Wikipedia but Wikipedia uses a lot of redirects, so multiple URIs can all ultimately point to the same page and therefore be used as a URL on another page. There's no guarantee that two page authors will use the same version of the name.

From this helpful article: https://towardsdatascience.com/calculating-string-similarity-in-python-276e18a7d33a

In [4]:
import string
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
def clean_string(text):
    text = text.replace("_", " ")
    #    text = "".join([word for word in text if word not in string.punctuation])
#    text = text.lower()
    return text

In [5]:
rawtravellers = c[(c["Type"]=="traveller")].index.to_list()
cleantravellers = list(map(clean_string, rawtravellers))
cleantravellers

['Abdul Ahad Mohmand',
 'Aidyn Aimbetov',
 'Akihiko Hoshide',
 'Alan Bean',
 'Alan G. Poindexter',
 'Alan Shepard',
 'Albert Sacco',
 'Aleksandr Ivanchenkov',
 'Aleksandr Kaleri',
 'Aleksandr Lazutkin',
 'Aleksandr Nikolayevich Balandin',
 'Aleksandr Panayotov Aleksandrov',
 'Aleksandr Pavlovich Aleksandrov',
 'Aleksandr Poleshchuk',
 'Aleksandr Samokutyayev',
 'Aleksandr Serebrov',
 'Aleksandr Skvortsov (cosmonaut)',
 'Aleksandr Viktorenko',
 'Aleksei Gubarev',
 'Aleksei Yeliseyev',
 'Aleksey Gubarev',
 'Aleksey Ovchinin',
 'Alexander Alexandrovich Volkov (cosmonaut)',
 'Alexander Gerst',
 'Alexander Laveykin',
 'Alexander Misurkin',
 'Alexander Viktorenko',
 'Alexei Leonov',
 'Alfred M. Worden',
 'Anatoli Artsebarsky',
 'Anatoli Brergovoy',
 'Anatoli Ivanishin',
 'Anatoli Levchenko',
 'Anatoly Filipchenko',
 'Anatoly Solovyev',
 'Andr%C3%A9 Kuipers',
 'Andreas Mogensen',
 'Andrei Borisenko',
 'Andrew J. Feustel',
 'Andrew M. Allen',
 'Andrew R. Morgan',
 'Andrew S. W. Thomas',
 'Andr

In [6]:
vectoriser = CountVectorizer().fit_transform(cleantravellers)
vectors = vectoriser.toarray()
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1]])

In [7]:
csim = cosine_similarity(vectors)
csim

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [8]:
dfcsim = pd.DataFrame(csim, index=rawtravellers, columns=rawtravellers)
dfcsim

,Abdul_Ahad_Mohmand,Aidyn_Aimbetov,Akihiko_Hoshide,Alan_Bean,Alan_G._Poindexter,Alan_Shepard,Albert_Sacco,Aleksandr_Ivanchenkov,Aleksandr_Kaleri,Aleksandr_Lazutkin,...,Yuri_Onufrienko,Yuri_Romanenko,Yuri_Shargin,Yuri_Usachev,Yuri_Usachyev,Yury_Malyshev_(cosmonaut),Yury_Usachev,Zhai_Zhigang,Zhang_Xiaoguang,Zvyozdochka
Abdul_Ahad_Mohmand,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
Aidyn_Aimbetov,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
Akihiko_Hoshide,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
Alan_Bean,0.0,0.0,0.0,1.0,0.5,0.5,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
Alan_G._Poindexter,0.0,0.0,0.0,0.5,1.0,0.5,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yury_Malyshev_(cosmonaut),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.000000,0.408248,0.0,0.0,0.0
Yury_Usachev,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.5,0.0,0.408248,1.000000,0.0,0.0,0.0
Zhai_Zhigang,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,1.0,0.0,0.0
Zhang_Xiaoguang,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,1.0,0.0


In [9]:
x = dfcsim.stack()
x = pd.DataFrame(x)
x.index.to_list()
y=[]
for t in x.index.to_list():
    y.append(t[0]==t[1])
y
x["SimpleMatch"] = y
x.rename(columns={0:"CosSimilar"},inplace=True)
x

CosSimilar  SimpleMatch
Abdul_Ahad_Mohmand Abdul_Ahad_Mohmand                1.0         True
                   Aidyn_Aimbetov                    0.0        False
                   Akihiko_Hoshide                   0.0        False
                   Alan_Bean                         0.0        False
                   Alan_G._Poindexter                0.0        False
...                                                  ...          ...
Zvyozdochka        Yury_Malyshev_(cosmonaut)         0.0        False
                   Yury_Usachev                      0.0        False
                   Zhai_Zhigang                      0.0        False
                   Zhang_Xiaoguang                   0.0        False
                   Zvyozdochka                       1.0         True

[355216 rows x 2 columns]

Filtering on the values... 1.0 means they're exactly the same, there will always be one of these (self). 2/3 seems to mean that they have a single name in common and throws up a lot. Using a threshold of just a little above 2/3 seems best.

In [10]:
similarNames = x[(x["CosSimilar"]>0.6)&(x["SimpleMatch"]==False)].sort_values(by="CosSimilar",ascending=False)
similarNames

,,CosSimilar,SimpleMatch
Gregory_C._Johnson,Gregory_H._Johnson,1.000000,False
Gregory_H._Johnson,Gregory_C._Johnson,1.000000,False
F%C3%A8i_J%C3%B9nl%C3%B3ng,J%C3%BCgderdemidiin_G%C3%BCrragchaa,0.707107,False
J%C3%BCgderdemidiin_G%C3%BCrragchaa,F%C3%A8i_J%C3%B9nl%C3%B3ng,0.707107,False
Claudie_Haigner%C3%A9,Jean-Pierre_Haigner%C3%A9,0.670820,False
Jean-Pierre_Haigner%C3%A9,Claudie_Haigner%C3%A9,0.670820,False
Aleksandr_Panayotov_Aleksandrov,Aleksandr_Pavlovich_Aleksandrov,0.666667,False
Scott_Kelly_(astronaut),James_M._Kelly_(astronaut),0.666667,False
Mark_Kelly_(astronaut),Scott_Kelly_(astronaut),0.666667,False
Michael_Collins_(astronaut),Michael_J._Smith_(astronaut),0.666667,False


Now to try out the links...

In [11]:
def wikilink(page):
    return '<a href="https://en.wikipedia.org/wiki/{}" target="wiki">{}</a>'.format(page,page)


similarNames.reset_index().style.format(wikilink)

,level_0,level_1,CosSimilar,SimpleMatch
0,Gregory_C._Johnson,Gregory_H._Johnson,0.9999999999999998,False
1,Gregory_H._Johnson,Gregory_C._Johnson,0.9999999999999998,False
2,F%C3%A8i_J%C3%B9nl%C3%B3ng,J%C3%BCgderdemidiin_G%C3%BCrragchaa,0.7071067811865477,False
3,J%C3%BCgderdemidiin_G%C3%BCrragchaa,F%C3%A8i_J%C3%B9nl%C3%B3ng,0.7071067811865477,False
4,Claudie_Haigner%C3%A9,Jean-Pierre_Haigner%C3%A9,0.6708203932499369,False
5,Jean-Pierre_Haigner%C3%A9,Claudie_Haigner%C3%A9,0.6708203932499369,False
6,Aleksandr_Panayotov_Aleksandrov,Aleksandr_Pavlovich_Aleksandrov,0.6666666666666669,False
7,Scott_Kelly_(astronaut),James_M._Kelly_(astronaut),0.6666666666666669,False
8,Mark_Kelly_(astronaut),Scott_Kelly_(astronaut),0.6666666666666669,False
9,Michael_Collins_(astronaut),Michael_J._Smith_(astronaut),0.6666666666666669,False


So with the exception of Gregory_C._Johnson and Gregory_H._Johnson who really do just have similar names, everything with a score above 0.7 is a genuine duplicate. Even James B  and James D. Wetherbee, who are actually the same person (according to wikipedia) - "Jim Wetherbee".

OK, so I did a bit of manual mucking about in a spreadsheet. There are only 30ish of them after all.

In [12]:
dups = pd.read_csv("data/duplicate_names.txt", sep="\t", header=0)
dups

,Replace,Preferred
0,Alan_B._Shepard,Alan_Shepard
1,Alan_L._Bean,Alan_Bean
2,Chris_A._Hadfield,Chris_Hadfield
3,Christopher_J._Cassidy,Christopher_Cassidy
4,Christopher_J._Ferguson,Christopher_Ferguson
5,David_R._Scott,David_Scott
6,Donald_R._Pettit,Donald_Pettit
7,Guion_S._Bluford,Guion_Bluford
8,James_B._Wetherbee,James_D._Wetherbee
9,James_D._A._van_Hoften,James_van_Hoften


In [13]:
el.replace(dups["Replace"].to_list(),dups["Preferred"].to_list(), inplace=True)

In [17]:
el[(el["subject"]=="Valeri_Polyokov")]

,date,subject,eventType,object


In [15]:
el.to_csv("data/sftl-deduped.txt", sep="\t", index=False)
